## NOTE, THIS NOTEBOOK RUNS ONLY ON DATABRICKS
If run without Databricks on Spark, Spark context etc. should be set

### 1. First setup the connection to S3 for data load

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [ ]:
access_key = dbutils.secrets.get(scope = "s3-access", key = "aws-access-key")
secret_key = dbutils.secrets.get(scope = "s3-access", key = "aws-secret-key")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# If you are using Auto Loader file notification mode to load files, provide the AWS Region ID.
aws_region = "us-west-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

s3_bucket = "s3a://ejkquant-uswest1/"

### 2. Then we load the intermediate results (bucketed data) from DBFS, join them with Datetime, and fill null values. Finally, we store the compelete train data set back to S3 for model training in later stages

In [ ]:
t1 = spark.table("MSFT")
t2 = spark.table("SP500")
t3 = spark.table("BOND")
t4 = spark.table("NOTE")
t5 = spark.table("GOLD")
t6 = spark.table("econ")

dataset = t1.join(t2, how="left", on="Datetime")
dataset = dataset.join(t3, how="left", on="Datetime")
dataset = dataset.join(t4, how="left", on="Datetime")
dataset = dataset.join(t5, how="left", on="Datetime")
dataset = dataset.join(t6, how="left", on="Datetime")

#dataset = forwardFillImputer(dataset, dataset.columns[1:3])
dataset = dataset.fillna(0)
dataset = dataset.withColumn("year", year(col("Datetime")))

dataset.write.format("parquet").partitionBy("year").mode("overwrite").save(s3_bucket+"train_data")